In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import datetime

In [39]:
# extract data in Excel file into dataframe
xls = pd.ExcelFile('Fitbit/2017-07.xls')
df2 = pd.read_excel(xls, 'Foods')

# 'dates' stores the dates (YYYY-MM-DD) that contain food entries
# 'food_logs' stores file names that contai food data
dates = []
food_logs = []

# if food entry is entered for that day, store file name and date into 
# their respective lists
for day in df2.values:
    if day[1] != '0':
        dates.append(day[0])
        foodLogSheetName = 'Food Log ' + day[0].replace('-', '')
        food_logs.append(foodLogSheetName)

# --------------------------------------------------------------

# dump Food Log sheet into a dataframe
# drop all rows that have all NaN values
df = pd.read_excel(xls, sheet_name='Food Log 20170706')
df.dropna(how='all', inplace=True)

# Forward fill for 'Meal' column -- CLEANING DATA / GETTING DATAFRAME TO LOOK NICE
df['Meal'] = df['Meal'].fillna(method='ffill')
df = df.reset_index(drop=True)

# stores index of data from 'df' for...
daily_total_index = []

# if the cell value in the 'Meal' column is 'Daily Totals', store the index of that row
for i in df.index:
    if df['Meal'][i] == 'Daily Totals':
        daily_total_index.append(i)

# create a DataFrame 'daily' to store macros information
# NOTE: 'Daily Totals' store info about a user's macros for the day
#       so should be placed in a separate DataFrame from food data
daily = pd.DataFrame()
daily = df.iloc[daily_total_index[1]:daily_total_index[-1]+1]
daily = daily.pivot(index='Meal', columns='Food', values='Calories')

# drop 'Daily Totals' data from 'df'
df = df.drop(index=daily_total_index)
daily

Food,Calories,Carbs,Fat,Fiber,Protein,Sodium,Water
Meal,,,,,,,
Daily Totals,699,93 g,26 g,16 g,32 g,612 mg,0 fl oz
